In [1]:
import datetime, urllib, ssl
import pandas as pd

In [ ]:
def UkrVHI_in_csv_for_area(index_area):
    ssl._create_default_https_context = ssl._create_unverified_context
    url = 'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2022&type=Mean'.format(index_area)
    wp = urllib.request.urlopen(url)
    text = wp.read()
    
    time_now = datetime.datetime.now()
    str_time_now = (str(time_now)).replace(' ', '_')
    str_time_now = (str_time_now).replace(':', '-')
    
    file_name = "VHI_id"+str(index_area)+"_"+str_time_now+".csv"
    
    out=open('UkrVHI_csv/'+file_name, 'wb')
    out.write(text)
    out.close()
    
    return((file_name, index_area))

In [ ]:
list_UkrVHI_files = []
for i in range(27):
    if (i!=11) and (i!=19):
        file_name, index_area = UkrVHI_in_csv_for_area(i+1)
        list_UkrVHI_files.append((file_name, index_area))
UkrVHI_files = pd.DataFrame(list_UkrVHI_files, columns=['file_name', 'index_area'])
UkrVHI_files.to_csv('UkrVHI_csv/info.csv')

In [10]:
df = pd.DataFrame()

path='UkrVHI_csv/'
UkrVHI_files = pd.read_csv(path+'info.csv',index_col=0)
name_columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']

for index in UkrVHI_files.index:
    temp = pd.read_csv(path+UkrVHI_files['file_name'][index],header=1,index_col=False,names=name_columns)
    temp = temp.assign(Area=UkrVHI_files['index_area'][index]) 
    df = pd.concat([df,temp],ignore_index=True)

df = df.dropna()
df = df.drop(df.loc[df['VHI']==-1].index)
df['Year']=df['Year'].replace(['<tt><pre>1982'],'1982')
df = df.astype({'Week':'int','Year':'int'})


In [3]:
area_ch = {1:22,2:24,3:23,4:25,5:3,6:4,7:8,8:19,9:20,10:21,11:9,13:10,14:11,15:12,16:13,17:14,18:15,19:16,21:17,22:18,23:6,24:1,25:2,26:7,27:5}

df['Area'].replace(area_ch,inplace=True)

In [4]:
def extremes_for_year(year,area):
    location = df.loc[(df['Area']==area)&(df['Year']==year)]
    minimum = min(location['VHI'])
    maximum = max(location['VHI'])
    print('For area',area,'in',year,': min(VHI) =',minimum,', max(VHI) =',maximum)
    
def extreme_drought(area):
    location = df.loc[(df['Area']==area)&(df['VHI']<15)]
    print('For area',area,'years with extreme drought:',location['Year'].unique())

def moderate_drought(area):
    location = df.loc[(df['Area']==area)&(df['VHI']<35)&(df['VHI']>=15)]
    print('For area',area,'years with moderate drought:',location['Year'].unique())

In [5]:
extremes_for_year(2021,5)

For area 5 in 2021 : min(VHI) = 36.11 , max(VHI) = 65.33


In [6]:
extreme_drought(20)

For area 20 years with extreme drought: [2003 2007]


In [7]:
moderate_drought(12)

For area 12 years with moderate drought: [1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995
 1996 2000 2001 2007 2013 2015 2019]
